In [4]:
#Load packages
from netCDF4 import Dataset
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import sys
import netCDF4 as nc4
from torch import nn
from torch import optim
import progressbar
import pytorch_lightning as pl
import torch.nn.functional as F
from torchvision import transforms
from torch import nn
from torch import optim
import progressbar
sys.path.append("/home2/datahome/tpicard/python/Python_Modules_p3_pyticles/")
import torch
from torch.utils.data import DataLoader, Dataset
from CNN_tools import *
from CNN_UNET import *
from pytorch_lightning.callbacks import ModelCheckpoint
from DATALOADER import Pdf_Image_DataSet, Pdf_Image_DataSet_image_process
from variables import *
import importlib

#  CNN TYPE ?

In [4]:
# Save direction and type of cnn
kernel_size=5
padding=2
bias=False
p_dropout=0
nlayer0=64

cnn_type = 'surf_1step'

if cnn_type == '4L':
    autoencoder = CNN_UNET_4L()
    filename_chkpt = 'CNN_UNET_4L_2.0'
elif cnn_type =='surf':
    
    autoencoder = CNN_UNET_SURF()
    filename_chkpt = 'CNN_UNET_SURF'
    
elif cnn_type =='surf_1step':
    
    autoencoder = CNN_UNET_SURF_1step()
    filename_chkpt = 'CNN_UNET_SURF_1STEP'
    
elif cnn_type =='generic':
    
    autoencoder = CNN_UNET_generic(kernel_size,padding,bias,p_dropout,nlayer0)
    filename_chkpt = 'CNN_UNET_k{0}_p{1}_b{2}_d{3}_nl{4}'.format(kernel_size,padding,bias,p_dropout,nlayer0)
    
dirSAVE = './Saved_model'
checkpoint_callback = ModelCheckpoint(monitor='loss_filter_200m_validation',
                                              dirpath= dirSAVE,
                                              filename= filename_chkpt + '-{epoch:02d}-{loss_filter_200m_validation:.2f}',
                                              save_top_k=2,
                                              mode='min')
dirLOG = './logs/{0}/'.format(filename_chkpt)

# IMPORT DATA

In [5]:
### LOAD DATA ####

#image_train = load_images_train('half')
image_train = load_images_train_surface('half')
Normalize = transforms.Normalize(list_mean_train_surface, list_std_train_surface)
image_train = Normalize(torch.tensor(image_train))

image_eval = load_images_validation_surface()
image_eval = Normalize(torch.tensor(image_eval))
#image_norm_train = load_images_train()

'''
# Normalization of inputs
Normalize = transforms.Normalize(list_mean_train, list_std_train)
image_train = Normalize(torch.tensor(image_train))

image_eval = load_images_validation()
image_eval = Normalize(torch.tensor(image_eval))
'''

pdf_train = load_pdf_train()
pdf_filter_train = load_pdf_filter_train()

pdf_train = load_pdf_train()
pdf_filter_train = load_pdf_filter_train()

pdf_train = np.transpose(pdf_train,(0,2,1,3,4))
pdf_train = pdf_train.reshape(pdf_train.shape[0]*pdf_train.shape[1],8,100,100)
pdf_train = pdf_train[::2]

pdf_filter_train = np.transpose(pdf_filter_train,(0,2,1,3,4))
pdf_filter_train = pdf_filter_train.reshape(pdf_filter_train.shape[0]*pdf_filter_train.shape[1],8,100,100)
pdf_filter_train = pdf_filter_train[::2]

#image_norm_eval = load_image_processed('validation')
#image_eval = load_images_validation()


pdf_eval = load_pdf_validation()
pdf_filter_eval = load_pdf_filter_validation()


pdf_eval = np.transpose(pdf_eval,(0,2,1,3,4))
pdf_eval = pdf_eval.reshape(pdf_eval.shape[0]*pdf_eval.shape[1],8,100,100)

pdf_filter_eval = np.transpose(pdf_filter_eval,(0,2,1,3,4))
pdf_filter_eval = pdf_filter_eval.reshape(pdf_filter_eval.shape[0]*pdf_filter_eval.shape[1],8,100,100)

Loading training surface data ...
Done
Loading validation surface data ...
Done


# CREATE TRAINING AND VALIDATION DATA

In [6]:
train_set = Pdf_Image_DataSet(image_train,pdf_train,pdf_filter_train,transform= ToTensor())
eval_set = Pdf_Image_DataSet(image_eval,pdf_eval,pdf_filter_eval,transform= ToTensor())

train_loader = DataLoader(train_set, batch_size=batch_size, num_workers = 0, shuffle = True, drop_last=False)
eval_loader = DataLoader(eval_set, batch_size=batch_size, num_workers = 0, shuffle = False, drop_last=False)


# TRAIN MODEL

In [7]:
# TRAINNING 

trainer = pl.Trainer(max_epochs=max_epochs,gpus=nb_gpus, default_root_dir=dirLOG, callbacks=[checkpoint_callback])
trainer.fit(model=autoencoder, train_dataloaders=train_loader,val_dataloaders=eval_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-312ca2e1-f08f-4fae-19ab-55f09735c2dc]
Missing logger folder: logs/CNN_UNET_SURF_1STEP/lightning_logs

  | Name      | Type       | Params
-----------------------------------------
0 | conv1     | Sequential | 90.2 K
1 | conv_up1  | Sequential | 115 K 
2 | softmax   | Softmax    | 0     
3 | flatten   | Flatten    | 0     
4 | maxpool2d | MaxPool2d  | 0     
5 | avgpool2d | AvgPool2d  | 0     
6 | relu      | ReLU       | 0     
-----------------------------------------
205 K     Trainable params
0         Non-trainable params
205 K     Total params
0.823     Total estimated model params size (MB)
/home2/datahome/tpicard/conda-env/croco/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /home2/datahome/tpicard/PhD_MOMOPAR/TRAIN_AND_VALIDATION_CNN/Saved_model exists and is no

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home2/datahome/tpicard/conda-env/croco/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home2/datahome/tpicard/conda-env/croco/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:  78%|███████▊  | 135/174 [00:09<00:02, 14.66it/s, loss=0.568, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 1:  78%|███████▊  | 136/174 [00:09<00:02, 14.44it/s, loss=0.553, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 2:  78%|███████▊  | 136/174 [00:09<00:02, 14.28it/s, loss=0.541, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 3:  78%|███████▊  | 136/174 [00:09<00:02, 14.37it/s, loss=0.542, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 4:  78%|███████▊  | 136/174 [00:09<00:02, 14.81it/s, loss=0.516, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 5:  78%|███████▊  | 136/174 [00:09<00:02, 14.83it/s, loss=0.518, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 6:  78%|███████▊  | 136/174 [00:09<00:02, 14.90it/s, loss=0.512, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 7:  78%|███████▊  | 136/174 [00:09<00:02, 14.81it/s, loss=0.509, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 8:  78%|███████▊  | 136/174 [00:09<00:02, 14.85it/s, loss=0.496, v_num=0]
Validating: 0it [00:00, ?it/s]
E

# TRAIN 10 MODELS

In [ ]:

for i in range(10):

    if cnn_type == '4L':
        autoencoder = CNN_UNET_4L()
        filename_chkpt = 'CNN_UNET_4L_{0}'.format(i)
    elif cnn_type =='surf':

        autoencoder = CNN_UNET_SURF()
        filename_chkpt = 'CNN_UNET_SURF_{0}'.format(i)
        
    filename_chkpt = 'CNN_UNET_SURFACE_{0}'.format(i)
    checkpoint_callback = ModelCheckpoint(monitor='loss_no_filter',
                                              dirpath= dirSAVE,
                                              filename= filename_chkpt,
                                              save_top_k=1,
                                              mode='min')

    dirLOG = './logs/supermodel/{0}'.format(filename_chkpt)


    # TRAINNING 
    trainer = pl.Trainer(max_epochs=max_epochs,gpus=nb_gpus, default_root_dir=dirLOG, callbacks=[checkpoint_callback])
    trainer.fit(model=autoencoder, train_dataloaders=train_loader,val_dataloaders=eval_loader)



In [4]:
#CNN using 4 vertical levels

alpha1 = 0.8
alpha2 = 1-0.8
nb_dx = 100

class CNN_UNET_4L(pl.LightningModule):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Sequential(    
        nn.Conv2d(76, 64, kernel_size = 3, padding = 1, bias=False),
        nn.ReLU(True),
        nn.BatchNorm2d(64),
        nn.Conv2d(64, 64, kernel_size = 3, padding = 1, bias=False),
        nn.BatchNorm2d(64)
             )
        
        self.conv2 = nn.Sequential(    
        nn.Conv2d(64, 128, kernel_size = 3, padding = 1, bias=False),
        nn.ReLU(True),
        nn.BatchNorm2d(128),
        nn.Conv2d(128, 128, kernel_size = 3, padding = 1, bias=False),
        nn.BatchNorm2d(128)
             )
        
        self.conv3 = nn.Sequential(    
        nn.Conv2d(128, 256, kernel_size = 3, padding = 1, bias=False),
        nn.ReLU(True),
        nn.BatchNorm2d(256),
        nn.Conv2d(256, 256, kernel_size = 3, padding = 1, bias=False),
        nn.BatchNorm2d(256)
             )
        
        self.conv_up1 = nn.Sequential(    
        nn.Conv2d(128, 64, kernel_size = 3, padding = 1, bias=False),
        nn.ReLU(True),
        nn.BatchNorm2d(64),
        nn.Conv2d(64, 64, kernel_size = 3, padding = 1, bias=False),
        nn.BatchNorm2d(64),
        nn.Conv2d(64, 8, kernel_size = 3, padding = 1, bias=False)
             )
        
        self.conv_up2 = nn.Sequential(    
        nn.Conv2d(256, 128, kernel_size = 3, padding = 1, bias=False),
        nn.ReLU(True),
        nn.BatchNorm2d(128),
        nn.Conv2d(128, 128, kernel_size = 3, padding = 1, bias=False),
        nn.BatchNorm2d(128)
             )
        
        self.convTrans3 = nn.ConvTranspose2d(256,128,kernel_size = 2,padding = 0,stride=2)
        self.convTrans2 = nn.ConvTranspose2d(128,64,kernel_size = 2,padding = 0,stride=2)
        
        self.softmax = nn.Softmax(dim=2)
        self.flatten = nn.Flatten(start_dim=2, end_dim=- 1)
        self.maxpool2d = nn.MaxPool2d(2)
        self.avgpool2d = nn.AvgPool2d(2)
        self.relu = nn.ReLU(True)
        
        
    def forward(self, z, y_filter,y):
        
        y_pred = torch.zeros(y.shape).to('cuda')
        #print(y_pred.device)
        y_pred[:,:,49:51,49:51] = 0.25 # Initialisation of y_pred
        #print(y_pred.device)
        #print(z.device)

        y1 = torch.cat((z,y_pred),dim=1) 
        y1 = self.conv1(y1)
        y2 = self.maxpool2d(y1)
        y2 = self.conv2(y2)
        y3 = self.maxpool2d(y2)
        y3 = self.conv3(y3)
        
        y3 = self.convTrans3(y3)
        y3 = torch.cat((y2,y3),dim=1)
        y2 = self.conv_up2(y3)
        
        y2 = self.convTrans2(y2)
        y2 = torch.cat((y1,y2),dim=1)
        y1 = self.conv_up1(y2)
        
        y_hat = self.flatten(y1)
        y_hat = self.relu(y_hat)
        y_hat = self.softmax(torch.log(y_hat+1e-10)) 
        y_hat = y_hat.view(y_hat.shape[0],8,nb_dx,nb_dx)
        
        return y_hat 
    
    def configure_optimizers(self):
        lr = 0.001
        optimizer = optim.Adam(self.parameters(),lr= lr, betas=(0.5, 0.999),weight_decay=0)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        
        z, y_f, y = batch
        y_hat = self(z,y_f,y)
        loss = 0

        
        for i in range(0,8):
            loss = loss + alpha1*Bhatta_loss(y_hat[:,i,:,:], y_f[:,i,:,:]) + alpha2*Bhatta_loss(y_hat[:,i,:,:], y[:,i,:,:])
            
        loss = loss / 8
        
        loss_filter_200m = Bhatta_loss(y_hat[:,-1,:,:], y_f[:,-1,:,:])
        
        self.log("loss_train", loss, on_epoch=True, on_step = True)
        self.log("loss_filter_200m_train", loss, on_epoch=True, on_step = True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        z, y_f, y = batch
        y_hat = self(z,y_f,y)
        
        loss_filter = 0
        loss_no_filter = 0

        for i in range(0,8):
            loss_filter = loss_filter + Bhatta_loss(y_hat[:,i,:,:], y_f[:,i,:,:])
            loss_no_filter = loss_no_filter + Bhatta_loss(y_hat[:,i,:,:], y[:,i,:,:])
            
        loss_filter = loss_filter / 8
        loss_no_filter = loss_no_filter / 8
        
        loss_filter_200m = Bhatta_loss(y_hat[:,-1,:,:], y_f[:,-1,:,:])
        
        self.log("loss_filter_validation", loss_filter, on_epoch=True, on_step = True)
        self.log("loss_no_filter_validation", loss_no_filter, on_epoch=True, on_step = True)
        self.log("loss_filter_200m_validation", loss_filter_200m, on_epoch=True, on_step = True)
        
        return loss_no_filter

In [25]:
# TRAINNING 
autoencoder = CNN_UNET_4L()
trainer = pl.Trainer(max_epochs=max_epochs,gpus=nb_gpus, default_root_dir=dirLOG, callbacks=[checkpoint_callback])
trainer.fit(model=autoencoder, train_dataloaders=train_loader,val_dataloaders=eval_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-dcb0ca9d-55d4-28a2-1aa7-2638e1a18e28]

   | Name       | Type            | Params
------------------------------------------------
0  | conv1      | Sequential      | 80.9 K
1  | conv2      | Sequential      | 221 K 
2  | conv3      | Sequential      | 885 K 
3  | conv_up1   | Sequential      | 115 K 
4  | conv_up2   | Sequential      | 442 K 
5  | convTrans3 | ConvTranspose2d | 131 K 
6  | convTrans2 | ConvTranspose2d | 32.8 K
7  | softmax    | Softmax         | 0     
8  | flatten    | Flatten         | 0     
9  | maxpool2d  | MaxPool2d       | 0     
10 | avgpool2d  | AvgPool2d       | 0     
11 | relu       | ReLU            | 0     
------------------------------------------------
1.9 M     Trainable params
0         Non-trainable params
1.9 M     Total params
7.643     Total estimated model params size (MB)


Epoch 0:  10%|▉         | 30/310 [00:09<01:27,  3.21it/s, loss=0.65, v_num=3] 

In [2]:


#Load packages
from netCDF4 import Dataset
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import sys
import netCDF4 as nc4
from torch import nn
from torch import optim
import progressbar
import pytorch_lightning as pl
import torch.nn.functional as F
from torchvision import transforms
from torch import nn
from torch import optim
import progressbar
sys.path.append("/home2/datahome/tpicard/python/Python_Modules_p3_pyticles/")
import torch
from torch.utils.data import DataLoader, Dataset

def read_input():
    nc_name = 'Inputs_vertical_levels_4layers_{0}_sorted.nc'.format(nb_dx)
    nc = nc4.Dataset('/home2/datawork/tpicard/Pyticles/CNN_DATA/BACKWARD_CORRECTION/{0}'.format(nc_name),'r')
    inputs_test = np.asfortranarray(nc.variables['inputs_test'])
    inputs_train = np.asfortranarray(nc.variables['inputs_train'])
    nc.close()
    return(inputs_test,inputs_train)

def read_output():
    
    if filt =='L50':
        nc_name = 'pdf_4levels_{0}_filter_L50_sorted.nc'.format(nb_dx)
        nc = nc4.Dataset('/home2/datawork/tpicard/Pyticles/CNN_DATA/BACKWARD_CORRECTION/{0}'.format(nc_name),'r')
        pdf_test = np.asfortranarray(nc.variables['pdf_test_filter'])
        pdf_train = np.asfortranarray(nc.variables['pdf_train_filter'])
        
    elif filt =='L5':
        nc_name = 'pdf_vertical_levels_{0}_filter_sorted.nc'.format(nb_dx)
        nc = nc4.Dataset('/home2/datawork/tpicard/Pyticles/CNN_DATA/BACKWARD_CORRECTION/{0}'.format(nc_name),'r')
        pdf_test = np.asfortranarray(nc.variables['pdf_test_filter'])
        pdf_train = np.asfortranarray(nc.variables['pdf_train_filter'])
        
    else :
        nc_name = 'pdf_vertical_levels_4layers{0}_sorted.nc'.format(nb_dx)
        nc = nc4.Dataset('/home2/datawork/tpicard/Pyticles/CNN_DATA/BACKWARD_CORRECTION/{0}'.format(nc_name),'r')
        pdf_test = np.asfortranarray(nc.variables['pdf_test'])*((800/nb_dx)**2)
        pdf_train = np.asfortranarray(nc.variables['pdf_train'])*((800/nb_dx)**2)
        nc.close()
        
    return(pdf_test,pdf_train)

class Pdf_Image_DataSet(Dataset):
    def __init__(self,images, pdf_f,pdf_nf,transform=None):
        
        self.pdf_f = pdf_f
        self.pdf_nf = pdf_nf
        self.images = images
        self.transform = transform

    def __len__(self):
        return self.pdf_f.shape[0]

    def __getitem__(self, idx):
        # select coordinates
        pdf_f_sample = self.pdf_f[idx,1:,0,:,:]
        pdf_nf_sample = self.pdf_nf[idx,1:,0,:,:]
        image_sample = self.images[idx,:,:,:]
        
        if self.transform:
            pdf_f_sample = self.transform(pdf_f_sample)
            pdf_nf_sample = self.transform(pdf_nf_sample)
            image_sample = self.transform(image_sample)
            
        return image_sample, pdf_f_sample, pdf_nf_sample

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    def __call__(self, sample):
        return torch.FloatTensor(sample)

#GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#READ DATA Filter
filt = 'L50'
(pdf_test_filter,pdf_train_filter) = read_output()
(inputs_test,inputs_train) = read_input()

#READ DATA No Filter
filt = 'No'
(pdf_test_no_f,pdf_train_no_f) = read_output()

# MEAN and STD for each channel
list_mean_train = []
list_std_train = []


for i in range(inputs_train.shape[1]):
    list_mean_train.append(np.mean(inputs_train[:,i,:,:]))
    list_std_train.append(np.std(inputs_train[:,i,:,:]))


# Normalization of inputs
Normalize = transforms.Normalize(list_mean_train, list_std_train)
inputs_test = Normalize(torch.tensor(inputs_test))
inputs_train = Normalize(torch.tensor(inputs_train))

i = 0

#inputs_test_1l = np.concatenate((inputs_test[:,4*i:4*i+4,:,:],inputs_test[:,4*i+16:4*i+4+16,:,:],inputs_test[:,4*i+32:4*i+4+32,:,:],inputs_test[:,4*i+48:4*i+4+48,:,:],inputs_test[:,64:,:,:]),axis=1)
#inputs_train_1l = np.concatenate((inputs_train[:,4*i:4*i+4,:,:],inputs_train[:,4*i+16:4*i+4+16,:,:],inputs_train[:,4*i+32:4*i+4+32,:,:],inputs_train[:,4*i+48:4*i+4+48,:,:],inputs_train[:,64:,:,:]),axis=1)



## reduce size dataset
train_set = Pdf_Image_DataSet(inputs_train,pdf_train_filter,pdf_train_no_f,transform= ToTensor())
train_loader_old = DataLoader(train_set, batch_size=batch_size, num_workers = 0, shuffle = True, drop_last=False)

test_set = Pdf_Image_DataSet(inputs_test,pdf_test_filter,pdf_test_no_f,transform= ToTensor())
test_loader_old = DataLoader(test_set, batch_size=batch_size, num_workers = 0, shuffle = False, drop_last=False)



In [4]:
inputs_train.shape

torch.Size([4900, 68, 100, 100])

In [5]:
pdf_train_filter.shape

(4900, 9, 1, 100, 100)

In [5]:
# TRAINNING 
autoencoder = CNN_UNET_4L()
trainer = pl.Trainer(max_epochs=max_epochs,gpus=nb_gpus, default_root_dir=dirLOG, callbacks=[checkpoint_callback])
trainer.fit(model=autoencoder, train_dataloaders=train_loader_old,val_dataloaders=test_loader_old)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-739dfd1b-f418-cc5b-11cf-7fb8737907b5]

   | Name       | Type            | Params
------------------------------------------------
0  | conv1      | Sequential      | 80.9 K
1  | conv2      | Sequential      | 221 K 
2  | conv3      | Sequential      | 885 K 
3  | conv_up1   | Sequential      | 115 K 
4  | conv_up2   | Sequential      | 442 K 
5  | convTrans3 | ConvTranspose2d | 131 K 
6  | convTrans2 | ConvTranspose2d | 32.8 K
7  | softmax    | Softmax         | 0     
8  | flatten    | Flatten         | 0     
9  | maxpool2d  | MaxPool2d       | 0     
10 | avgpool2d  | AvgPool2d       | 0     
11 | relu       | ReLU            | 0     
------------------------------------------------
1.9 M     Trainable params
0         Non-trainable params
1.9 M     Total params
7.643     Total estimated model params size (MB)
/home2/datahome/tpicar

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home2/datahome/tpicard/conda-env/croco/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home2/datahome/tpicard/conda-env/croco/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:  91%|█████████ | 154/170 [00:35<00:03,  4.29it/s, loss=0.447, v_num=6]
Validating: 0it [00:00, ?it/s]
Epoch 1:  91%|█████████ | 154/170 [00:35<00:03,  4.31it/s, loss=0.342, v_num=6]
Validating: 0it [00:00, ?it/s]
Epoch 2:  91%|█████████ | 154/170 [00:36<00:03,  4.27it/s, loss=0.254, v_num=6]
Validating: 0it [00:00, ?it/s]
Epoch 3:  21%|██        | 36/170 [00:08<00:31,  4.31it/s, loss=0.247, v_num=6] 

/home2/datahome/tpicard/conda-env/croco/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Epoch 3:  21%|██        | 36/170 [00:25<01:35,  1.40it/s, loss=0.247, v_num=6]

In [1]:
from torchsummary import summary
summary(autoencoder)

ModuleNotFoundError: No module named 'torchsummary'

In [9]:
pytorch_total_params = sum(p.numel() for p in CNN_UNET_SURF().parameters())
print(pytorch_total_params)

1883072


In [6]:
pytorch_total_params

1910720